In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy.sparse import csc_matrix, csr_matrix, hstack
import time
import numpy as np
import pickle

from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import shuffle
from keras.preprocessing import text, sequence
from sklearn.metrics import log_loss,roc_auc_score
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_pickle(FILE.train_ori.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_ori.value)
print('test shape is: {}'.format(test.shape))

# X = pd.concat([train.drop(['click'],axis=1),test])
X = pd.concat([train,test],sort=False)
print(X.shape)


X_shiyi = pd.read_pickle(FILE.shiyi_fillna_ori.value)
print(X_shiyi.shape)

X = X.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')
print(X.shape)
ignore_columns = ['instance_id','time','click'] + ['creative_is_js', 'creative_is_voicead', 'app_paid']

train shape is: (1001650, 35)
test shape is: (40024, 34)
(1041674, 35)
(1041674, 45)
(1041674, 36)


In [3]:
# need_process_col = list(set(X.columns) - set(ignore_columns))
# X_ = X[need_process_col].copy()
# counter = 0
# for col in tqdm(X_.columns):
# #     X_[col] = le.fit_transform(X_[col].astype(str))
#     X_[col] = col + '_'+X_[col].astype(str)
#     nunique = X_[col].nunique()

In [4]:
need_process_col = list(set(X.columns) - set(ignore_columns))
X_ = X[need_process_col].copy()
counter = 0
for col in tqdm(X_.columns):
    X_[col] = le.fit_transform(X_[col].astype(str))
    X_[col] = col + '_'+X_[col].astype(str)

100%|██████████| 30/30 [01:41<00:00,  3.39s/it]


In [5]:
train_index = pickle.load(open(FILE.train_index.value,'rb'))
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))

In [6]:
val_length = 150000
train_length = 1001650

train_split, val_split, y_train, y_val = train_test_split(X_.iloc[:train_length],train.click.values,test_size = 0.1, shuffle=True)

maxlen = 1

# y_train = train['click'].values[:-val_length]
# y_val = train['click'].values[-val_length:]
cols = need_process_col
tok_dict = {}
input_train_dict = {}
input_val_dict = {}
input_test_dict = {}
for col in tqdm(cols):
    tok=text.Tokenizer(num_words=X_[col].nunique(),lower=False,filters='!')
    tok.fit_on_texts(list(X_[col]))
    tok_dict[col] = tok
    ###################################################################################################
#     X_train=tok.texts_to_sequences(list(X_[col].iloc[:train_length-val_length]))
#     X_val=tok.texts_to_sequences(list(X_[col].iloc[train_length-val_length:train_length]))
    ###################################################################################################
    
    X_train=tok.texts_to_sequences(list(train_split[col].values))
    X_val=tok.texts_to_sequences(list(val_split[col].values))
    X_test=tok.texts_to_sequences(list(X_[col].values[train_length:]))
    ###################################################################################################
    x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
    x_val=sequence.pad_sequences(X_val,maxlen=maxlen)
    x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
    input_train_dict['input_'+col] = x_train
    input_val_dict['input_'+col] = x_val
    input_test_dict['input_'+col] = x_test

100%|██████████| 30/30 [05:45<00:00, 11.51s/it]


# Build NN model only use model

In [7]:

def get_nn_model(cols):
    input_list = []
    flatten_list = []
    for col in cols:
        max_feature = X_[col].nunique()
        cur_input = Input(shape=(1, ),name = 'input_'+col)
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 3:
            embed_size = 3
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=1,
                            trainable=True,
#                                 0.0004
                            embeddings_regularizer=l2(0.0004),
                            name='ebd_'+col)(cur_input)
        embed_layer = SpatialDropout1D(0.5)(embed_layer)
        x = Flatten()(embed_layer)
        input_list.append(cur_input)
        flatten_list.append(x)
    x = concatenate(flatten_list)
    
    x = Dense(5124, activation='relu')(x)
    x = Dropout(0.2)(x)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.2)(x)

    preds = Dense(1, activation="sigmoid")(x)
    model = Model(input_list, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
    return model

In [8]:
model = get_nn_model(cols)
tolerance = 2
cur_to = 0
best_logloss = None
best_weights = None
while True:
    model.fit(input_train_dict, y_train, 
              batch_size=5000, 
              epochs=1,
              verbose=1,
              shuffle=True,
              )
    preds = model.predict(input_val_dict,5000,verbose=1)
    logloss = log_loss(y_val,preds)
    roc = roc_auc_score(y_val,preds)
    print(logloss)
    print(roc)
    if best_logloss is None:
        best_logloss = logloss
        best_weights = model.get_weights()
    else:
        if best_logloss > logloss:
            best_logloss = logloss
            best_weights = model.get_weights()
            cur_to = 0
        else:
            cur_to +=1
    if cur_to == 3:
        break
    print('best logloss is: {}'.format(best_logloss))
model.set_weights(best_weights)

Epoch 1/1
100165/100165 [==============================] - 1s 10us/step
0.4203874662395839
0.7635568981550331
best logloss is: 0.4203874662395839
Epoch 1/1
100165/100165 [==============================] - 0s 5us/step
0.41895112837801113
0.7636666845981331
best logloss is: 0.41895112837801113
Epoch 1/1
100165/100165 [==============================] - 1s 5us/step
0.4183461695158708
0.7649116464437378
best logloss is: 0.4183461695158708
Epoch 1/1
100165/100165 [==============================] - 1s 6us/step
0.41848632453354273
0.7648336642229738
best logloss is: 0.4183461695158708
Epoch 1/1
100165/100165 [==============================] - 0s 5us/step
0.4182409781702095
0.7655862537961091
best logloss is: 0.4182409781702095
Epoch 1/1
100165/100165 [==============================] - 1s 6us/step
0.4184931413222091
0.7654615250537429
best logloss is: 0.4182409781702095
Epoch 1/1
100165/100165 [==============================] - 0s 4us/step
0.41781966656599917
0.7663708741271593
best logloss is:

In [9]:
aaa

NameError: name 'aaa' is not defined

In [ ]:
preds = model.predict(input_test_dict,5000,verbose=1)

In [ ]:
save = test[['instance_id']].copy()
save['predicted_score'] = preds
save.to_csv('submitnn_ebd_ini.csv',index=False)

In [ ]:
col = 'model'
ebd_mat = model.get_layer('ebd_{}'.format(col)).get_weights()[0]
print('ebd_mat shape is: {}'.format(ebd_mat.shape))
seq = tok_dict[col]
tseq = seq.texts_to_sequences(X_[col])
pseq = sequence.pad_sequences(tseq).squeeze()
extract = ebd_mat[pseq]

In [ ]:
extract_list = []
for col in tqdm(need_process_col):
    ebd_mat = model.get_layer('ebd_{}'.format(col)).get_weights()[0]
    seq = tok_dict[col]
    tseq = seq.texts_to_sequences(X_[col])
    pseq = sequence.pad_sequences(tseq).squeeze()
    extract = ebd_mat[pseq]
    extract_list.append(extract)

In [ ]:
X_matrix = np.concatenate(extract_list,axis=1)
print(X_matrix.shape)

In [ ]:
pickle.dump(X_matrix,open(FILE.X_fe_emd_format.value.format('col{}_hasUtag'.format(X_matrix.shape[1])),'wb'))